In [36]:
from scipy.stats import chisquare

test = 6306
control = 6043
total_trafic = test + control

observed = [test, control]
expected =  [total_trafic / 2, total_trafic /2]

chi = chisquare(observed, f_exp = expected)

print (chi)

if chi[1] < 0.01:
	print("SRM is present  = НЕСООТВЕТСВИЕ ПОЛЬЗОВАТЕЛЕЙ ГРУПП ОЖИДАЕМЫМ")
else:
	print("SRM is not present НЕТ НЕСООТВЕТСВИЯ ПОЛЬЗОВАТЕЛЕЙ ГРУПП ОЖИДАЕМЫМ")

Power_divergenceResult(statistic=5.601182281966151, pvalue=0.017948361469529415)
SRM is not present НЕТ НЕСООТВЕТСВИЯ ПОЛЬЗОВАТЕЛЕЙ ГРУПП ОЖИДАЕМЫМ


In [41]:
import pandas as pd

df = pd.read_excel('/content/contracts_plus_loses_dash_0409_со столбцом аномалий (2).xlsx', sheet_name='testcontrol', index_col=0)

In [42]:
control = df[df['group'] == 'control']
test = df[df['group'] == 'test']

In [ ]:
control.drop('group', axis= 1 , inplace= True )
test.drop('group', axis= 1 , inplace= True )
control.drop('experiment_id', axis= 1 , inplace= True )
test.drop('experiment_id', axis= 1 , inplace= True )

In [44]:
print(ttest_ind(control, test, equal_var=False))

print('Средняя потеря по пользователю контрольная группа', control.mean())
print('Средняя потеря по пользователю тестовая группа', test.mean())

TtestResult(statistic=array([-0.34528481]), pvalue=array([0.72988769]), df=array([9771.57595476]))
Средняя потеря по пользователю контрольная группа loss    876.690844
dtype: float64
Средняя потеря по пользователю тестовая группа loss    1080.716454
dtype: float64


In [ ]:
# 72%>5% следованиельно на уровне значимости 95% не можем отвергнуть нулевую гипотезу,
#следовательно средняя потеря по полисам не изменилось после внедрения нового подхода

In [45]:
def p_bin(m1,n1,m2,n2,level_conf=0.05):
    # на вход ф-ии подается кол-во успехов варианта А и общее число вариантов m1,n1
    #и число успехов варанта Б и общее число вариантов m2, n2,
    #а также уровень значимости - опционально (0.05 по умолчанию)
    from scipy import stats
    T = (m1/n1 - m2/n2)/((m1+m2)/(n1+n2)*(1 - (m1+m2)/(n1+n2))*(1/n1+1/n2))**0.5 #формула биномиального критерия вер-сти
    P = 1 - stats.norm.cdf(T)  # вычисляем p-значение
    # критичная область у нас со знаком БОЛЬШЕ, потому что задачался вопрос в альтернативной что ВАРИАНТ А был более значимым
    print(f"Stats {T}, p-value {P}")
    if (P < level_conf):
        print(f"На уровне значимости {level_conf} нулевая гипотеза отвергается в пользу альтернативной: Вариаент А более значим (значимая разница Варианта А перед вариантом Б есть)")
    else:
        print(f"На уровне значимости {level_conf} нулевая гипотеза принимается (нет значимой разницы между вариантом А и Б)")
#p_bin(46,1100,30,1000,0.1) Пример ф-иии и ответа:
# На уровне значимости 0.1 нулевая гипотеза отвергается в пользу альтернативной: Вариаент А более значим (значимая разница Варианта А перед вариантом Б есть)

In [47]:
control_not0 = control[control['loss'] != 0].count()
test_not0 = test[test['loss'] != 0].count()


print('Средняя потеря по пользователю контрольная группа', control_not0)
print('Средняя потеря по пользователю тестовая группа', test_not0)

Средняя потеря по пользователю контрольная группа loss    30
dtype: int64
Средняя потеря по пользователю тестовая группа loss    14
dtype: int64


In [48]:
p_bin(control_not0, control.count(),test_not0,test.count())

Stats loss    2.558502
dtype: float64, p-value [0.00525621]
На уровне значимости 0.05 нулевая гипотеза отвергается в пользу альтернативной: Вариаент А более значим (значимая разница Варианта А перед вариантом Б есть)


In [49]:
# Итого по метрике "кол-ву потерь" разница значимая, а по сумме потерь - значимой разницы нет.